# Visualising flaring data in timeseries


In [51]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots

base_dir = os.path.dirname(os.getcwd())

In [2]:
kuristan_faring_monthly_total = pd.read_csv(f"{base_dir}/summarised_data/kurdistan_processed_data/annual_flaring_count.csv")
iraq_faring_monthly_total = pd.read_csv(f"{base_dir}/summarised_data/iraq_processed_data/annual_flaring_count.csv")

In [3]:
iraq_faring_monthly_total.rename(
    columns=({'Count': 'Iraq_count'}), 
    inplace=True,
)
kuristan_faring_monthly_total.rename(
    columns=({'Count': 'Kurdistan_count'}), 
    inplace=True,
)

In [4]:
iraq_kurdistan_faring_monthly_total = pd.merge(kuristan_faring_monthly_total, iraq_faring_monthly_total, on='Flaring_month')

In [48]:
iraq_kurdistan_faring_monthly_total

,Flaring_month,Kurdistan_count,Iraq_count
0,2018-10,4503,30450
1,2018-11,4490,34101
2,2018-12,6338,28668
3,2019-01,6880,40032
4,2019-02,4351,29174
5,2019-03,6661,29270
6,2019-04,6998,40666
7,2019-05,8489,50779
8,2019-06,10621,37224
9,2019-08,11158,43454


In [56]:
# Visualising monthly changes in flaring in kurdistan and Iraq

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=iraq_kurdistan_faring_monthly_total['Flaring_month'], y=iraq_kurdistan_faring_monthly_total['Iraq_count'], name="Monthly count of flares Iraq"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=iraq_kurdistan_faring_monthly_total['Flaring_month'], y=iraq_kurdistan_faring_monthly_total['Kurdistan_count'], name="Monthly count of flares Kurdistan"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Count of flaring in Iraq and Kurdistan "
)

# Set x-axis title
fig.update_xaxes(title_text="Month")

fig.add_vline(x=datetime.datetime.strptime("2021-07-13", "%Y-%m-%d").timestamp() * 1000, annotation_text="Start date for  <br> flaring phase-out" )

# Set y-axes titles
fig.update_yaxes(title_text="<b>Iraqi</b> flaring count", secondary_y=False)
fig.update_yaxes(title_text="<b>Kurdistani</b> flaring count", secondary_y=True)

fig.show()

In [44]:
iraq_data_dir = f"{base_dir}/summarised_data/iraq_processed_data"
kurdistan_data_dir = f"{base_dir}/summarised_data/kurdistan_processed_data"

def create_flaring_by_date_df(data_dir):
    df_list = []
    for dirname,  _, filelist in os.walk(data_dir):
        not_wanted_filenames = ["annual_flaring_count.csv"]

        flaring_filepaths = [
            os.path.join(dirname, filename) for filename in filelist
            if not any(filetype in filename for filetype in not_wanted_filenames)
        ]

        for filename in flaring_filepaths:
            flaring_by_date = pd.read_csv(os.path.join(dirname, filename))

            df_list.append(flaring_by_date)
    return df_list

kurdistan_total_flaring_by_date = pd.concat(create_flaring_by_date_df(kurdistan_data_dir))   
iraq_total_flaring_by_date = pd.concat(create_flaring_by_date_df(iraq_data_dir))   


In [45]:
kurdistan_total_flaring_by_date = kurdistan_total_flaring_by_date.sort_values(by="Flaring_date")
iraq_total_flaring_by_date = iraq_total_flaring_by_date.sort_values(by="Flaring_date")

In [47]:
# Visualising monthly changes in flaring in kurdistan and Iraq

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=iraq_total_flaring_by_date['Flaring_date'], y=iraq_total_flaring_by_date['Count'], name="Count of flares Iraq"),
    secondary_y=True,
)

# Add traces
fig.add_trace(
    go.Scatter(x=kurdistan_total_flaring_by_date['Flaring_date'], y=kurdistan_total_flaring_by_date['Count'], name="Count of flares Kurdistan"),
    secondary_y=False,
)

# Add figure title
fig.update_layout(
    title_text="Count of flaring in Iraq and Kurdistan"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Kurdistani</b> flaring count", secondary_y=False)
fig.update_yaxes(title_text="<b>Iraqi</b> flaring count", secondary_y=True)

fig.show()